In [1]:
from collections import OrderedDict
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import sampler
import os
import torchvision.datasets as dset
import torchvision.transforms as T
import pandas as pd
# for plotting
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

from trainer_functions.cifartrainer import evaluate_cifar, train_cifar, build_cifar10
from models.ResNet import ResNetCifar as ResNet
from trainer_functions.cifarCloader import CIFARCDataset

/home/sheikh/miniconda3/envs/cnn/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/sheikh/miniconda3/envs/cnn/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# empty dataframe
df = pd.DataFrame(columns=['corruption', 'accuracy', "lr", "state", "n_subset"])

curruption_list = ["brightness", "contrast", "defocus_blur", "elastic_transform", "fog", "frost", "gaussian_blur", "gaussian_noise", "glass_blur",
 "impulse_noise", "jpeg_compression", "motion_blur", "pixelate", "saturate", "shot_noise", "snow", "spatter", "speckle_noise", "zoom_blur"]

n_subset = 3000
lrs = [1e-3, 1e-4, 1e-5]

In [4]:
cifarc = CIFARCDataset(corruption="brightness")
ccdataloader = DataLoader(cifarc, batch_size=64, shuffle=True)

cctrain = torch.utils.data.Subset(cifarc, range(0, n_subset))
ccvalid = torch.utils.data.Subset(cifarc, range(6000, 8000))
cctest = torch.utils.data.Subset(cifarc, range(8000, 10000))

cctrainload = DataLoader(cctrain, batch_size=64, shuffle=True)

In [10]:
images, labels = next(iter(ccdataloader))
# plot this images and labels
print(images.shape)
#plt.imshow(images[0].permute(1, 2, 0))
labels


torch.Size([64, 3, 32, 32])


tensor([9, 0, 0, 6, 4, 2, 8, 5, 2, 4, 6, 2, 5, 6, 8, 3, 7, 1, 2, 4, 3, 8, 5, 8,
        2, 0, 1, 8, 8, 9, 8, 1, 2, 3, 7, 5, 1, 3, 1, 0, 7, 2, 5, 4, 1, 6, 6, 6,
        5, 6, 0, 3, 1, 7, 3, 8, 9, 3, 0, 2, 5, 2, 3, 6], dtype=torch.uint8)

## writing fine tuning code here

In [9]:
for corruption in curruption_list:
    for lr in lrs:
        cifarc = CIFARCDataset(corruption=corruption)
        ccdataloader = DataLoader(cifarc, batch_size=64, shuffle=True)

        cctrain = torch.utils.data.Subset(cifarc, range(0, n_subset))
        ccvalid = torch.utils.data.Subset(cifarc, range(6000, 8000))
        cctest = torch.utils.data.Subset(cifarc, range(8000, 10000))

        cctrainload = DataLoader(cctrain, batch_size=64, shuffle=True)
        ccvalidload = DataLoader(ccvalid, batch_size=64, shuffle=True)
        cctestload = DataLoader(cctest, batch_size=64, shuffle=True)


        device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

        tune_net = build_cifar10(device)

        optimizer = optim.Adam(tune_net.layer2.parameters(), lr=lr, weight_decay=0.0001) # change the optimizer to tune different layers

        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1) # no need to modify this

        checkpoint = torch.load('checkpoints/ckpt.pth')
        tune_net.load_state_dict(checkpoint['net'])


        acc = train_cifar(tune_net, cctrainload, ccvalidload, optimizer, scheduler, device=device, epochs=15)

        # adding results to dataframe 
        dict = {}
        dict['corruption'] = corruption
        dict['accuracy'] = acc
        dict['lr'] = lr
        dict['state'] = "layer2"
        dict['n_subset'] = n_subset
        df_temp = pd.DataFrame(dict, index=[0])
        df = pd.concat([df, df_temp]) 

# saves the results
df.to_csv("cifarC_layer2_3000.csv")        

Building model...
Epoch 0, loss = 0.6436, lr 0.00100000
Got 1441 / 2000 correct (72.05)

Epoch 1, loss = 0.2523, lr 0.00100000
Got 1616 / 2000 correct (80.80)

Epoch 2, loss = 0.0668, lr 0.00100000
Got 1661 / 2000 correct (83.05)

Epoch 3, loss = 0.0102, lr 0.00100000
Got 1649 / 2000 correct (82.45)

Epoch 4, loss = 0.0092, lr 0.00100000
Got 1673 / 2000 correct (83.65)

Epoch 5, loss = 0.0032, lr 0.00100000
Got 1672 / 2000 correct (83.60)

Epoch 6, loss = 0.0025, lr 0.00100000
Got 1682 / 2000 correct (84.10)

Epoch 7, loss = 0.0023, lr 0.00100000
Got 1687 / 2000 correct (84.35)

Epoch 8, loss = 0.0017, lr 0.00100000
Got 1684 / 2000 correct (84.20)

Epoch 9, loss = 0.0022, lr 0.00100000
Got 1691 / 2000 correct (84.55)

Epoch 10, loss = 0.0020, lr 0.00100000
Got 1690 / 2000 correct (84.50)

Epoch 11, loss = 0.0017, lr 0.00100000
Got 1692 / 2000 correct (84.60)

Epoch 12, loss = 0.0011, lr 0.00100000
Got 1697 / 2000 correct (84.85)

Epoch 13, loss = 0.0015, lr 0.00100000
Got 1693 / 2000 c

In [12]:
# saves the results
name = "cifarC" + dict['state'] + str(n_subset) + ".csv"
name

'cifarCfcn1000.csv'

In [11]:
tune_net

ResNetCifar(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (bn1): GroupNorm(8, 16, eps=1e-05, affine=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): GroupNorm(8, 16, eps=1e-05, affine=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): BasicBlock(
      (bn1): GroupNorm(8, 16, eps=1e-05, affine=True)
      (relu1): ReLU(inplace=True)
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): GroupNorm(8, 16, eps=1e-05, affine=True)
      (relu2): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (2): BasicBlock(
      (bn1): GroupNorm(8, 16, eps=1e-05, affine=True)
      (relu1): ReLU(in